In [74]:
from collections import deque
import pandas as pd
from haversine import haversine
from haversine import haversine_vector, Unit
import math
import numpy as np

def encontrar_melhor_rota(grafo, inicio, destino):
    fila = deque([(inicio, [inicio], 0)])
    melhor_rota = None
    menor_custo = float('inf')
    while fila:
        vertice, rota_atual, custo_atual = fila.popleft()
        if vertice == destino:
            if custo_atual < menor_custo:
                melhor_rota = rota_atual
                menor_custo = custo_atual
        if vertice in grafo:
            for proximo, peso in grafo[vertice].items():
                if proximo not in rota_atual:
                    fila.append((proximo, rota_atual + [proximo], custo_atual + peso))
    return melhor_rota, menor_custo

def get_inicial_point(user_location, df_routes):
    points = {
        key: tuple(map(float, value.split(", "))) 
        for key, value in df_routes[["Cordenadas", "Nome"]].set_index("Nome").to_dict()["Cordenadas"].items()
    }
    for key, value in points.items():
        points[key] = haversine(user_location, value, unit=Unit.METERS)
    return min(points, key=points.get)

def get_possibilidades(df_routes):
    routes_temp = df_routes[["Cordenadas", "Nome", "Acesso"]].set_index("Nome").to_dict()
    routes_possibilidades = {}
    config_cord = lambda cord: tuple(map(float, cord.split(", ")))
    for acesso_cordenada, cordenadas in routes_temp["Cordenadas"].items():
        routes_possibilidades[acesso_cordenada] = {}
        for acesso_caminho in routes_temp["Acesso"][acesso_cordenada].split(", "):
            routes_possibilidades[acesso_cordenada][acesso_caminho] = haversine(
                config_cord(cordenadas), 
                config_cord(routes_temp["Cordenadas"][acesso_caminho]
            ), unit=Unit.METERS)
    return routes_possibilidades

In [57]:
user_location = (-23.6698337,-46.7010033)
df_routes = pd.read_excel(r"C:\Users\Lucas\Desktop\EasyWay\routes_file\teste.xlsx")

df_routes_points = df_routes[["Cordenadas", "Nome"]]
points = {key: tuple(map(float, value.split(", "))) for key, value in df_routes[["Cordenadas", "Nome"]].set_index("Nome").to_dict()["Cordenadas"].items()}

In [77]:
possibilidades_com_pesos = get_possibilidades(df_routes)
inicio = get_inicial_point(user_location, df_routes)
destino = "Sala teste 1"
print(encontrar_melhor_rota(possibilidades_com_pesos, inicio, destino))


(['Entrada', 'Entrada A', 'A', 'B', 'B1', 'Sala teste 1'], 162.98864947610735)
